In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e2/sample_submission.csv
/kaggle/input/playground-series-s6e2/train.csv
/kaggle/input/playground-series-s6e2/test.csv


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

In [3]:
base_dir = "/kaggle/input/playground-series-s6e2/"
train = pd.read_csv(f"{base_dir}train.csv")
test = pd.read_csv(f"{base_dir}test.csv")

In [4]:
train.head(10)

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence
5,5,38,1,4,138,283,0,0,147,1,1.6,2,2,7,Presence
6,6,59,1,4,130,246,0,2,152,0,0.8,2,2,3,Presence
7,7,60,0,3,120,245,0,0,151,0,1.2,1,0,3,Absence
8,8,48,0,4,140,212,0,2,125,0,0.0,1,0,3,Absence
9,9,44,0,4,150,197,0,0,150,0,0.0,2,0,3,Absence


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       630000 non-null  int64  
 1   Age                      630000 non-null  int64  
 2   Sex                      630000 non-null  int64  
 3   Chest pain type          630000 non-null  int64  
 4   BP                       630000 non-null  int64  
 5   Cholesterol              630000 non-null  int64  
 6   FBS over 120             630000 non-null  int64  
 7   EKG results              630000 non-null  int64  
 8   Max HR                   630000 non-null  int64  
 9   Exercise angina          630000 non-null  int64  
 10  ST depression            630000 non-null  float64
 11  Slope of ST              630000 non-null  int64  
 12  Number of vessels fluro  630000 non-null  int64  
 13  Thallium                 630000 non-null  int64  
 14  Hear

In [6]:
train['y'] = train['Heart Disease'].map({'Presence' : 1, 'Absence' : 0})

In [7]:
X_test = test
X = train.drop(columns=['Heart Disease', 'y'])
y = train['y']

# Train models

In [8]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
scale_pos_weight = (y == 0).sum() / (y == 1).sum()

y_probs_val = np.zeros(len(X))   # OOF validation predictions for entire dataset
y_probs = np.zeros(len(X_test))  # averaged test set predictions

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"\nTraining fold {fold + 1}/{n_splits} >>>")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    xgb_clf = XGBClassifier(
        # tree_method='hist',
        # n_estimators=1000,
        # objective='binary:logistic',
        random_state=42,
        # enable_categorical=True,
        # eval_metric='auc',
        # booster='gbtree',
        n_jobs=-1,
        # reg_lambda=4.510522889747622,
        # reg_alpha=5.007953193043952, 
        # colsample_bytree=0.5831655543160346,
        # subsample=0.9808690492838653,
        # learning_rate=0.008247101477015132,
        # max_depth=11,
        # min_child_weight=1,
        # device='cuda',
        # scale_pos_weight=scale_pos_weight  # Added parameter to handle class imbalance
    )

    xgb_clf.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        # early_stopping_rounds=100,
        verbose=500
    )

    # Store fold validation predictions in the correct place
    y_probs_val[val_idx] = xgb_clf.predict_proba(X_val)[:, 1]

    # Accumulate test set predictions, averaged over folds
    y_probs += xgb_clf.predict_proba(X_test)[:, 1] / n_splits


Training fold 1/5 >>>
[0]	validation_0-logloss:0.52938
[99]	validation_0-logloss:0.26931

Training fold 2/5 >>>
[0]	validation_0-logloss:0.53031
[99]	validation_0-logloss:0.27222

Training fold 3/5 >>>
[0]	validation_0-logloss:0.53013
[99]	validation_0-logloss:0.26995

Training fold 4/5 >>>
[0]	validation_0-logloss:0.53022
[99]	validation_0-logloss:0.27125

Training fold 5/5 >>>
[0]	validation_0-logloss:0.53003
[99]	validation_0-logloss:0.26860


In [9]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

y_probs_lgb = np.zeros(len(X_test))          # For averaged test set predictions
y_probs_val_lgb = np.zeros(len(X))           # For out-of-fold validation predictions

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}/{n_splits} >>>")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    lgb_clf = lgb.LGBMClassifier(
        # n_estimators=1000,
        # learning_rate=0.06,
        # num_leaves=100,
        # max_depth=10,
        # min_child_samples=9,
        # subsample=0.8,
        # colsample_bytree=0.5,
        # reg_alpha=0.79,
        # reg_lambda=3.0,
        # max_bin=4523,
        random_state=42,
        # verbosity=-1,
        # class_weight='balanced'  # Added parameter to handle class imbalance

    )
    
    lgb_clf.fit(
        X_train, 
        y_train, 
        eval_set=[(X_val, y_val)], 
        callbacks=[
            lgb.early_stopping(100),
            lgb.log_evaluation(period=500)
        ]
    )

    # Save validation fold predictions into the correct positions of full OOF array
    y_probs_val_lgb[val_idx] = lgb_clf.predict_proba(X_val)[:, 1]

    # Accumulate test predictions averaged over folds
    y_probs_lgb += lgb_clf.predict_proba(X_test)[:, 1] / n_splits

Training fold 1/5 >>>
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 225963, number of negative: 278037
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034757 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 14
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448339 -> initscore=-0.207383
[LightGBM] [Info] Start training from score -0.207383
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.269151
Training fold 2/5 >>>
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[Ligh

In [10]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize arrays for OOF validation and test predictions
y_probs_val_cat = np.zeros(len(X))       # full-length OOF array for training data
y_probs_cat = np.zeros(len(X_test))      # averaged test set predictions

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}/{n_splits} >>>")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)
    test_pool = Pool(X_test)

    cat_clf = CatBoostClassifier(
        random_state=42,
        # early_stopping_rounds=100,
        # eval_metric="Logloss",
        # n_estimators=5000,
        # learning_rate=0.06524873965257823,
        # l2_leaf_reg=0.8867612905712001,
        # bagging_temperature=0.1317347791955057,
        # random_strength=0.9922857768340815,
        # depth=7,
        # min_data_in_leaf=8,
        # task_type="CPU",
        # verbose=500, # Log every 100 iterations,
        # auto_class_weights='Balanced'
    )

    # Fit the model first!
    cat_clf.fit(train_pool, eval_set=val_pool, use_best_model=True)

    # Predict on validation fold and assign to correct positions
    y_probs_val_cat[val_idx] = cat_clf.predict_proba(val_pool)[:, 1]

    # Accumulate test set predictions averaged over folds
    y_probs_cat += cat_clf.predict_proba(test_pool)[:, 1] / n_splits

Training fold 1/5 >>>
Learning rate set to 0.14737
0:	learn: 0.5503059	test: 0.5498810	best: 0.5498810 (0)	total: 129ms	remaining: 2m 8s
1:	learn: 0.4621045	test: 0.4613450	best: 0.4613450 (1)	total: 191ms	remaining: 1m 35s
2:	learn: 0.4061780	test: 0.4051589	best: 0.4051589 (2)	total: 251ms	remaining: 1m 23s
3:	learn: 0.3703781	test: 0.3692690	best: 0.3692690 (3)	total: 310ms	remaining: 1m 17s
4:	learn: 0.3460128	test: 0.3448500	best: 0.3448500 (4)	total: 368ms	remaining: 1m 13s
5:	learn: 0.3291944	test: 0.3278030	best: 0.3278030 (5)	total: 432ms	remaining: 1m 11s
6:	learn: 0.3173152	test: 0.3158444	best: 0.3158444 (6)	total: 499ms	remaining: 1m 10s
7:	learn: 0.3091207	test: 0.3076588	best: 0.3076588 (7)	total: 560ms	remaining: 1m 9s
8:	learn: 0.3025988	test: 0.3010797	best: 0.3010797 (8)	total: 620ms	remaining: 1m 8s
9:	learn: 0.2972200	test: 0.2958229	best: 0.2958229 (9)	total: 678ms	remaining: 1m 7s
10:	learn: 0.2928703	test: 0.2914180	best: 0.2914180 (10)	total: 739ms	remaining: 1

In [11]:
oof_auc_xgb = roc_auc_score(y, y_probs_val)
oof_auc_lgb = roc_auc_score(y, y_probs_val_lgb)
oof_auc_cat = roc_auc_score(y, y_probs_val_cat)
print(f"XGBoost OOF AUC: {oof_auc_xgb:.5f}")
print(f"LightGBM OOF AUC: {oof_auc_lgb:.5f}")
print(f"CatBoost OOF AUC: {oof_auc_cat:.5f}")

XGBoost OOF AUC: 0.95464
LightGBM OOF AUC: 0.95473
CatBoost OOF AUC: 0.95538


In [12]:
xgb_pred = xgb_clf.predict_proba(X_test)[:,1]
print(xgb_pred)

lgb_pred = lgb_clf.predict_proba(X_test)[:,1]
print(lgb_pred)

cat_pred = cat_clf.predict_proba(X_test)[:,1]
print(cat_pred)

[0.9294335  0.00817715 0.992361   ... 0.03894913 0.19871075 0.01065035]
[0.90996705 0.01872774 0.97685876 ... 0.05673907 0.23202546 0.02493368]
[0.91922113 0.00771512 0.98785298 ... 0.05501766 0.16756144 0.01621255]


In [13]:
def submit(df, pred, fname):
    submission = pd.DataFrame({'id' : df['id'], 'y':pred})
    submission = submission.sort_values(by="y", ascending=True)
    submission.to_csv(fname, index=False)
    
submit(test, xgb_pred, 'submission_xgb.csv')
submit(test, lgb_pred, 'submission_lgb.csv')
submit(test, cat_pred, 'submission_cat.csv')